In [ ]:
import sys
import numpy as np
from PyQt5.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QPushButton, QLabel,
    QFileDialog, QProgressBar, QFrame, QCheckBox
)
from PyQt5.QtGui import QPixmap, QFont
from PyQt5.QtCore import Qt
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, AveragePooling2D,
    Dropout, Flatten, Dense
)
from PIL import Image


def build_meso4():
    model = Sequential()
    model.add(Conv2D(8, (3, 3), padding='same', activation='relu', input_shape=(256, 256, 3)))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Conv2D(8, (5, 5), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Conv2D(16, (5, 5), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(2, 2), padding='same'))
    model.add(Conv2D(16, (5, 5), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(AveragePooling2D(pool_size=(4, 4), padding='same'))
    model.add(Flatten())
    model.add(Dropout(0.5))
    model.add(Dense(16))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model


def preprocess_image(img_path, target_size=(256, 256)):
    img = Image.open(img_path).convert("RGB")
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0
    return np.expand_dims(img_array, axis=0)


def predict_with_new_model(img_path):
    img = preprocess_image(img_path)
    return new_model.predict(img, verbose=0)[0][0]


def predict_with_meso_model(img_path):
    img = preprocess_image(img_path)
    return meso_model.predict(img, verbose=0)[0][0]


new_model = load_model("/Users/kshitijverma/Downloads/new_model.h5")
meso_model = build_meso4()
meso_model.load_weights("/Users/kshitijverma/MesoNet/weights/Meso4_DF.h5")


class FakeImageDetectorApp(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Fake Image Detection")
        self.setGeometry(200, 100, 550, 550)

        self.layout = QVBoxLayout()
        self.layout.setAlignment(Qt.AlignCenter)
        self.setStyleSheet("background-color: #2b2b2b; color: white;")

        self.image_frame = QFrame()
        self.image_frame.setStyleSheet("""
            QFrame {
                border: 2px solid #555;
                border-radius: 10px;
                background-color: #3a3a3a;
            }
        """)
        self.image_frame.setFixedSize(280, 280)

        self.image_label = QLabel("No Image Selected", self.image_frame)
        self.image_label.setAlignment(Qt.AlignCenter)
        self.image_label.setStyleSheet("color: #aaa;")
        self.image_label.setGeometry(10, 10, 260, 260)
        self.layout.addWidget(self.image_frame, alignment=Qt.AlignCenter)

        self.progress_bar = QProgressBar(self)
        self.progress_bar.setRange(0, 100)
        self.progress_bar.setValue(0)
        self.progress_bar.setFixedWidth(300)
        self.progress_bar.setStyleSheet("""
            QProgressBar {
                border: 2px solid #555;
                border-radius: 5px;
                text-align: center;
                color: white;
                background-color: #3a3a3a;
            }
            QProgressBar::chunk {
                background-color: #00e676;
                width: 20px;
            }
        """)
        self.layout.addWidget(self.progress_bar, alignment=Qt.AlignCenter)

        self.conf_toggle = QCheckBox("Show Confidence Details")
        self.conf_toggle.setChecked(True)
        self.conf_toggle.setStyleSheet("""
            QCheckBox {
                color: white;
                font-size: 14px;
            }
        """)
        self.layout.addWidget(self.conf_toggle, alignment=Qt.AlignCenter)

        self.result_label = QLabel("")
        self.result_label.setFont(QFont("Arial", 13))
        self.result_label.setAlignment(Qt.AlignCenter)
        self.layout.addWidget(self.result_label)

        self.button = QPushButton("Select Image")
        self.button.setStyleSheet("""
            QPushButton {
                background-color: #00e676;
                color: black;
                border-radius: 8px;
                font-size: 15px;
                padding: 8px 16px;
                font-weight: bold;
            }
            QPushButton:hover {
                background-color: #00c853;
            }
        """)
        self.button.clicked.connect(self.open_file_dialog)
        self.layout.addWidget(self.button, alignment=Qt.AlignCenter)

        self.setLayout(self.layout)
        self.image_path = None

    def open_file_dialog(self):
        file_path, _ = QFileDialog.getOpenFileName(
            self, "Select Image", "", "Image Files (*.png *.jpg *.jpeg)"
        )
        if file_path:
            self.image_path = file_path
            pixmap = QPixmap(self.image_path).scaled(260, 260, Qt.KeepAspectRatio, Qt.SmoothTransformation)
            self.image_label.setPixmap(pixmap)
            self.predict_image()

    def predict_image(self):
        if not self.image_path:
            self.result_label.setText("Please select an image.")
            return

        try:
            self.progress_bar.setValue(10)
            pred_new = predict_with_new_model(self.image_path)
            self.progress_bar.setValue(50)

            pred_meso = predict_with_meso_model(self.image_path)
            self.progress_bar.setValue(80)

            avg_pred = (pred_new + pred_meso) / 2
            label = "Fake" if avg_pred > 0.5 else "Real"
            confidence = avg_pred if avg_pred > 0.5 else 1 - avg_pred

            base_text = f"<b>Final Decision:</b> {label} ({confidence*100:.2f}% confident)"
            if self.conf_toggle.isChecked():
                base_text += (
                    f"<br><br>New Model: {'Fake' if pred_new>0.5 else 'Real'} ({pred_new*100:.2f}%)"
                    f"<br>Meso4 Model: {'Fake' if pred_meso>0.5 else 'Real'} ({pred_meso*100:.2f}%)"
                )

            color = "#ff5252" if label == "Fake" else "#69f0ae"
            self.result_label.setStyleSheet(f"color: {color}; font-size: 15px;")
            self.result_label.setText(base_text)
            self.progress_bar.setValue(100)

        except Exception as e:
            self.result_label.setStyleSheet("color: #ff5252;")
            self.result_label.setText(f"Error: {str(e)}")
            self.progress_bar.setValue(0)


if __name__ == '__main__':
    app = QApplication(sys.argv)
    window = FakeImageDetectorApp()
    window.show()
    sys.exit(app.exec_())
